In [46]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.nn.functional as F
import numpy as np

In [47]:
class LR(nn.Module):
    def __init__(self):
        super(LR,self).__init__()
        self.fc = nn.Linear(24,2)
    def forward(self,x):
        out =  torch.sigmoid(self.fc(x))
        return out

In [48]:
data = np.loadtxt("data/german.data-numeric")

In [49]:
"""
标准化数据
"""
row,col = data.shape
for c in range(col-1):
    std = np.std(data[:,c])
    mean = np.mean(data[:,c])
    data[:,c] = (data[:,c] - mean) / std
data[:,col-1] =  data[:,col-1] - 1

In [50]:
np.random.shuffle(data)
x_train,x_test,y_train,y_test = data[:900,:24], data[900:,:24], data[:900,24], data[900:,24]
x_train = torch.from_numpy(x_train).float()
y_train = torch.from_numpy(y_train).long()
x_test = torch.from_numpy(x_test).float()
y_test = torch.from_numpy(y_test).long()

In [51]:
# prepare
net = LR()
criterion = nn.CrossEntropyLoss()
optimier=torch.optim.Adam(net.parameters()) # Adam优化

In [52]:
def test(pred,lab):
    t=pred.max(-1)[1]==lab
    return torch.mean(t.float())

In [53]:
print(x_test.size())

torch.Size([100, 24])


In [54]:
for i in range(2000):
    # print(criterion())
    net.train()
    y_hat = net(x_train)
   
    loss=criterion(y_hat,y_train)
    
    optimier.zero_grad()
    loss.backward()
    optimier.step()
    
    if i % 100  == 0:
#         net.eval()
        print(loss,test(net(x_test),y_test))

tensor(0.7233, grad_fn=<NllLossBackward>) tensor(0.4200)
tensor(0.6623, grad_fn=<NllLossBackward>) tensor(0.6700)
tensor(0.6253, grad_fn=<NllLossBackward>) tensor(0.7100)
tensor(0.6037, grad_fn=<NllLossBackward>) tensor(0.7100)
tensor(0.5894, grad_fn=<NllLossBackward>) tensor(0.7400)
tensor(0.5790, grad_fn=<NllLossBackward>) tensor(0.7300)
tensor(0.5710, grad_fn=<NllLossBackward>) tensor(0.7300)
tensor(0.5646, grad_fn=<NllLossBackward>) tensor(0.7300)
tensor(0.5593, grad_fn=<NllLossBackward>) tensor(0.7200)
tensor(0.5549, grad_fn=<NllLossBackward>) tensor(0.7200)
tensor(0.5512, grad_fn=<NllLossBackward>) tensor(0.7200)
tensor(0.5479, grad_fn=<NllLossBackward>) tensor(0.7300)
tensor(0.5451, grad_fn=<NllLossBackward>) tensor(0.7300)
tensor(0.5425, grad_fn=<NllLossBackward>) tensor(0.7300)
tensor(0.5403, grad_fn=<NllLossBackward>) tensor(0.7400)
tensor(0.5383, grad_fn=<NllLossBackward>) tensor(0.7400)
tensor(0.5365, grad_fn=<NllLossBackward>) tensor(0.7300)
tensor(0.5348, grad_fn=<NllLoss

## 对于eval 和 train的解释
+ 使用PyTorch进行训练和测试时一定注意要把实例化的model指定train/eval，eval（）时，框架会自动把BN和DropOut固定住，不会取平均，而是用训练好的值，不然的话，一旦test的batch_size过小，很容易就会被BN层导致生成图片颜色失真极大